# About the Notebook
 *  <a href="#Introduction">Introduction</a>
*  <a href="#EDA">EDA</a>
    *  Pivot Tables
    *  Box Plots and Violin Charts
    

    

# Introduction
We are provided with two datasets, namely -  
Los Angeles County Restaurant And Market Inspections :  
https://data.lacounty.gov/Health/LOS-ANGELES-COUNTY-RESTAURANT-AND-MARKET-INSPECTIO/6ni6-h5kp  
Los Angeles County Restaurant And Market Violations :   
https://data.lacounty.gov/Health/LOS-ANGELES-COUNTY-RESTAURANT-AND-MARKET-VIOLATION/8jyd-4pv9  

By going throught the above links, we can find that the Violations dataset itslef contains information about Inspections data as well, in th datasets provided by Kaggle.
([Link for Kaggle Data Set](http://www.kaggle.com/meganrisdal/la-county-restaurant-inspections-and-violations/home ))


### Importing Libraries and Datasets

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
inspections = pd.read_csv("../input/restaurant-and-market-health-inspections.csv")
violations = pd.read_csv("../input/restaurant-and-market-health-violations.csv")

Let`s inspect both the datasets and have a look on the both of the datasets samples.  
### Inspections

In [ ]:
inspections.info()

In [ ]:
inspections.head(5)

### Vioalations

In [ ]:
violations.info()

In [ ]:
violations.head(5)

We will start exploring Violation datasets from here - 

### PE Description  
Describes the type of program.  
From the below table we get to know that, this feild can be further expanded to more categorries like, Type, Seating Strength, and Risk Type which can help in more detailing.
Moreover,  4 out of 18 program type, constitute more than 60% of the total dataset, in which **RESTAURANT (0-30) SEATS HIGH RISK** is just more than 25%.

In [ ]:
pd.DataFrame(round(violations['pe_description'].value_counts(normalize=True)*100,2))

### Program Status
Indicates the status of the program, categories into Active and Inactive in active constitute around 88 % of all dataset.

In [ ]:
pd.DataFrame(round(violations['program_status'].value_counts(normalize=True)*100,2))

### Service Description
Describes the service that was performed. We see that most of the inspection held were Routined one (around 98%)

In [ ]:
pd.DataFrame(round(violations['service_description'].value_counts(normalize=True)*100,2))

### Violation Status
Identifies the status of the violation. Most prevailing Violation STatus is **Out of Compliane**, nearly 100%

In [ ]:
pd.DataFrame(round(violations['violation_status'].value_counts(normalize=True)*100,4))

**Points** : The amount of points deducted from the score for a violation.  
**Score** : The score of the inspection.  
So, we can create a new column, which will give us the net effective score for each inspection.  
**Net Effective Score** : Score - Points  
We'll also create a new column for Year, although this can be extracted by converting the **activity date** column to datetype, but for the time we'll extract the first four characters of the same column, which in result will give us the Year of the Inspection


In [ ]:
violations['Net Effective Score'] = violations['score'] - violations['points']
violations['Year'] = violations['activity_date'].str[:4]

# EDA
Grouping data on Violation Status and seet what is the average net effective score by year. We'll dig into each violation status in further steps.

In [ ]:
violations_group = pd.pivot_table(violations, values='Net Effective Score', index=[ 'violation_status'],
                    columns=['Year'], aggfunc=np.mean)
pd.DataFrame(violations_group.to_records())

As noticed earlier and also confirming from above results, we need to filter data based on vioaltion status and then analyzed the individual datasets.The interesting case to analyse will be **OUT OF COMPLIANCE** violation status.

In [ ]:
vio_Housing = violations[violations['violation_status'] == "HOUSING NON-CRITICAL"]
vio_Compliance = violations[violations['violation_status'] == "OUT OF COMPLIANCE"]
vio_Violation = violations[violations['violation_status'] == "VIOLATION"]

We cna first, find the net effective score aggregated by multiple variables for Violation Status,  Housing Non-Critical and Violation, as they each constitute to 0.0004%  of all dataset and then will dig into Violation related to Compliance. 

### Housing Non-Critical

In [ ]:
vio_Housing_group = pd.pivot_table(vio_Housing, values='Net Effective Score', index=['pe_description','violation_description','facility_name','facility_address','owner_name','grade','program_status','service_description','score','points'],
                    columns=['Year'], aggfunc=np.mean)
pd.DataFrame(vio_Housing_group.to_records())

### Vioaltion

In [ ]:
vio_Violation_group = pd.pivot_table(vio_Violation, values='Net Effective Score', index=['pe_description','violation_description','facility_name','facility_address','owner_name','grade','program_status','service_description','score','points'],
                    columns=['Year'], aggfunc=np.mean)
pd.DataFrame(vio_Violation_group.to_records())

# Vioaltion Status = Complaince 

We'll first visualize the top 4 categories in PE Description, which are frequently occuring in the dataset.The following table shows the top 4 categories in the dataset.Hence, we'll filter the our dataset for Violation Status = Complaince on these four categories


In [ ]:
pd.DataFrame(round(violations['pe_description'].value_counts(normalize=True)*100,2)).head(4)

In [ ]:
box1 = vio_Compliance[(vio_Compliance['pe_description'] == "RESTAURANT (0-30) SEATS HIGH RISK")|
                      (vio_Compliance['pe_description'] == "RESTAURANT (31-60) SEATS HIGH RISK") |
                      (vio_Compliance['pe_description'] == "RESTAURANT (0-30) SEATS MODERATE RISK") |
                      (vio_Compliance['pe_description'] == "RESTAURANT (61-150) SEATS HIGH RISK")]

## Box Plot - Net Effective Score for each selected category in pe_description by Year
The following graph will shoe us the trend in Net effective score in these categories by Year. One can have an idea about the five-number summary for each category within a year about the same.

In [ ]:
sns.set()
fig, ax = plt.subplots()
fig.set_size_inches(15, 7)

ax = sns.boxplot(x='pe_description', y='Net Effective Score',hue="Year", data=box1)
ax.set_xlabel("pe_description",fontsize=15)
ax.set_ylabel("Net Effective Score",fontsize=15)
ax.tick_params(labelsize=13)
plt.xticks(rotation=9)



## Violin Charts - Net Effective Score for each selected categories in pe_description by Year
We can see that from below graph that, Grade C Restaurant/Market have less net effective score, followed by Grade B which in results followed by Grade A, which has the highest among the all.

In [ ]:
sns.set()
fig, ax = plt.subplots()
fig.set_size_inches(15, 7)

ax = sns.violinplot(x='pe_description', y='Net Effective Score',hue="grade", data=box1)
ax.set_xlabel("pe_description",fontsize=15)
ax.set_ylabel("Net Effective Score",fontsize=15)
ax.tick_params(labelsize=13)
plt.xticks(rotation=9)



## Grouping by PE Description and calculation the net effective score by year for each underlying category

In [ ]:
pd.set_option("display.max_rows", 20)
vio_Compliance_peDes_group = pd.pivot_table(vio_Compliance, values='Net Effective Score', index=['pe_description'],columns=['Year'], aggfunc=np.mean)
vio_Compliance_peDes = pd.DataFrame(vio_Compliance_peDes_group.to_records())
vio_Compliance_peDes = pd.melt(vio_Compliance_peDes, id_vars='pe_description', value_vars=vio_Compliance_peDes.columns.drop('pe_description')).rename(columns={"variable":"Year", "value":"Avg. Net Effective Score"})
vio_Compliance_peDes

### Line Plot - Trend in average effective score by pe_descriptiob by year

In [ ]:
sns.set_style("ticks")
fig, ax = plt.subplots()
fig.set_size_inches(15, 6)

ax = sns.lineplot(x="pe_description", y="Avg. Net Effective Score",hue="Year",palette=["r", "b","g","y"], data=vio_Compliance_peDes)
ax.set_xlabel("pe_description",fontsize=15)
ax.set_ylabel("Avg. Net Effective Score",fontsize=15)
ax.grid(True)
ax.tick_params(labelsize=13)
plt.xticks(rotation=-270)

## Box Plot - For each category in pe_description, a box plot for net effective score by Year, for trend.

In [ ]:
sns.set()

g = sns.FacetGrid(vio_Compliance, col='pe_description',col_wrap=3,height=4, aspect=1.5)
g = g.map(sns.boxplot, "Year", "Net Effective Score")


## Violin Chart - A trend of net effective score by year within a grade

In [ ]:
sns.set()

g = sns.FacetGrid(vio_Compliance, col='grade',col_wrap=3,height=8, aspect=0.6)
g = g.map(sns.violinplot, "Year", "Net Effective Score")

## Grouping by Violation Description and calculating the average net effective score by year for each underlying category
Showing only first 10 rows and last 10 rows.

In [ ]:
pd.set_option("display.max_rows", 20)
vio_Compliance_VioDes_group = pd.pivot_table(vio_Compliance, values='Net Effective Score', index=['violation_description'],columns=['Year'], aggfunc=np.mean)
vio_Compliance_VioDes = pd.DataFrame(vio_Compliance_VioDes_group.to_records())
vio_Compliance_VioDes

## The following tables shows the Name of the facility in each Year, by its Grade in each Violation Descrption with :

### Maximum Net Effective Score 

In [ ]:
max_score =vio_Compliance.loc[vio_Compliance.groupby(['Year','violation_description'])['Net Effective Score'].idxmax()]

In [ ]:
pd.set_option("display.max_rows", 40)
pd.pivot_table(max_score, values='Net Effective Score', index=['violation_description','Year','grade','facility_name'])

###  Minimum Net Effective Score

In [ ]:
min_score =vio_Compliance.loc[vio_Compliance.groupby(['Year','violation_description'])['Net Effective Score'].idxmin()]

In [ ]:
pd.set_option("display.max_rows", 40)
pd.pivot_table(min_score, values='Net Effective Score', index=['violation_description','Year','grade','facility_name'])